In [2]:
%matplotlib inline

import datacube
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from deafrica_tools.plotting import display_map, rgb
from deafrica_tools.datahandling import load_ard, mostcommon_crs, wofs_fuser


In [3]:
dc = datacube.Datacube(app="Monitoring_water_quality")

In [4]:
lat_range = (15.65018332, 15.95028652)
lon_range = (80.75094696, 81.05003692)
product_name = "s2a_sen2cor_granule"
time_range = ('2022-01-15', '2023-02-15')
display_map(lon_range, lat_range)

/home/adithya/anaconda3/envs/odc_env/lib/python3.8/site-packages/deafrica_tools/plotting.py:284: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  all_longitude, all_latitude = transform(Proj(crs), Proj("EPSG:4326"), all_x, all_y)


In [5]:
ds = dc.load(product="s2a_sen2cor_granule",
                 measurements=["red","green","blue", "nir", "B11_20m"],
             x=lon_range,
             y=lat_range,
             time=time_range,
             output_crs='EPSG:6933',
             resolution=(-30, 30))
print(ds)

<xarray.Dataset>
Dimensions:      (time: 2, y: 1230, x: 963)
Coordinates:
  * time         (time) datetime64[ns] 2022-08-23T05:14:41.977841 2023-02-09T...
  * y            (y) float64 2.009e+06 2.009e+06 ... 1.972e+06 1.972e+06
  * x            (x) float64 7.791e+06 7.791e+06 7.791e+06 ... 7.82e+06 7.82e+06
    spatial_ref  int32 6933
Data variables:
    red          (time, y, x) uint16 1764 2057 2108 2057 ... 1154 1139 1150 1163
    green        (time, y, x) uint16 1990 2013 1988 2055 ... 1229 1237 1263 1269
    blue         (time, y, x) uint16 1721 1804 1827 1798 ... 1266 1287 1287 1290
    nir          (time, y, x) uint16 2855 2936 2502 2705 ... 1104 1101 1094 1101
    B11_20m      (time, y, x) uint16 2149 2014 1796 1920 ... 1076 1092 1080 1081
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref


In [9]:
# Load water observations
water = dc.load(
            product="s2a_sen2cor_granule",
            group_by="solar_day",
            like=ds.geobox,
            dask_chunks={"time": 1, "x": 2000, "y": 2000},
            time=time_range).water

#extract from mask the areas classified as water
water_extent = (water == 128).squeeze()

print(water_extent)

AttributeError: 'NoneType' object has no attribute 'begin'